In [ ]:
%pip show webdriver-manager
%pip show selenium

In [1]:
import json
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import StaleElementReferenceException, TimeoutException


In [33]:
def fetch_connected_cases(driver, case_data):
    table_row_path = ".//div[@id='w3-tab1']//table[contains(@class,'kv-grid-table')]/tbody/tr"
    history_rows = driver.find_elements(By.XPATH, table_row_path)

    history_list = []
    for hist_row in history_rows:
        hist_cells = hist_row.find_elements(By.TAG_NAME, "td")
        
        # skip "No results found"
        if len(hist_cells) == 1 and "No results found" in hist_cells[0].text:
            continue
        
        if len(hist_cells) >= 5:
            history_data = {
                "sr_number": hist_cells[0].text,
                "case_no": hist_cells[1].text,
                "case_title": hist_cells[2].text,
                "last_hearing": hist_cells[3].text,
                "next_date": hist_cells[4].text,
            }
            history_list.append(history_data)
            
    case_data["details"]["connected_cases"] = history_list



# def fetch_connected_cases(driver, case_data):
#     # wait until the connected cases tab-pane is active
#     active_tab = WebDriverWait(driver, 5).until(
#         EC.presence_of_element_located(
#             (By.XPATH, "//div[contains(@class,'tab-pane') and contains(@class,'active')]")
#         )
#     )

#     # now find rows only inside that tab
#     history_rows = active_tab.find_elements(By.XPATH, ".//tbody/tr")
#     print("TAB HTML:", active_tab.get_attribute("outerHTML")[:500])

#     history_list = []
#     for hist_row in history_rows:
#         hist_cells = hist_row.find_elements(By.TAG_NAME, "td")

#         # skip "No results found"
#         if len(hist_cells) == 1 and "No results found" in hist_cells[0].text:
#             continue

#         if len(hist_cells) >= 5:
#             history_data = {
#                 "sr_number": hist_cells[0].text.strip(),
#                 "case_no": hist_cells[1].text.strip(),
#                 "case_title": hist_cells[2].text.strip(),
#                 "last_hearing": hist_cells[3].text.strip(),
#                 "next_date": hist_cells[4].text.strip(),
#             }
#             history_list.append(history_data)

#     case_data["details"]["connected_cases"] = history_list


In [34]:
def fetch_case_history(driver, case_data):
    table_row_path = "//table[@class='kv-grid-table table table-bordered table-striped table-condensed']/tbody/tr"
    history_rows = driver.find_elements(By.XPATH, table_row_path)
    
    history_list = []
    for hist_row in history_rows:
        hist_cells = hist_row.find_elements(By.TAG_NAME, "td")
        if len(hist_cells) > 1:
            history_data = {
                "date": hist_cells[0].text,
                "list": hist_cells[1].text,
                "sr_number": hist_cells[2].text,
                "cma": hist_cells[3].text,
                "stage": hist_cells[6].text if len(hist_cells) > 6 else "",
                "bench": hist_cells[7].text if len(hist_cells) > 7 else "",
                "status": hist_cells[8].text if len(hist_cells) > 8 else "",
            }
            history_list.append(history_data)
    case_data["details"]["case_history"] = history_list

In [35]:

def fetch_misc_appln(driver, data):
    pass
def fetch_case_advocates(driver, data):
    pass
def fetch_parties_details(driver, data):
    pass

In [36]:
def per_page_extraction(driver, data):
    table_rows_path = "//table/tbody/tr"
    table_rows = driver.find_elements(By.XPATH, table_rows_path)
    
    for row_idx in range(len(table_rows)):
        # re-fetch rows every iteration (to avoid stale element reference)
        table_rows = driver.find_elements(By.XPATH, table_rows_path)
        row = table_rows[row_idx]

        cells = row.find_elements(By.TAG_NAME, "td")
        if not cells or len(cells) < 12:
            continue
        
        case_data = {
            "serial_number": cells[0].text,
            "case_catagory": cells[1].text,
            "case_number": cells[2].text,
            "case_year": cells[3].text,
            "bench": cells[4].text,
            "curcuit_code": cells[5].text,
            "case_title": cells[6].text,
            "matter/summary": cells[7].text,
            "last_hearing": cells[8].text,
            "next_date": cells[9].text,
            "disposal_date": cells[10].text,
            "status": cells[11].text,
            "details": {
                "connected_cases": [],
                "misc_appln_cma": [],
                "case_advocates": [],
                "parties_details": [],
                "case_history": []
            }
        }
        print("herer1")
        
        action_btn = row.find_element(By.CLASS_NAME, "btn")
        action_btn.click()
        print("herer2")

        # wait for modal
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, "printable"))
        )
        print("herer3")

        # subsection tabs
        section_handlers = {
            "Case History": fetch_case_history,
            "Connected Cases": fetch_connected_cases,
            "Misc Appln (CMA)": fetch_misc_appln,
            "Case Advocates": fetch_case_advocates,
            "Parties Details": fetch_parties_details,
        }
        path_to_subsections = "//div[@class='tabs-x  tabs-above tab-align-left tabs-krajee']/ul/li/a"

        for section, handler in section_handlers.items():
            try:
                # always re-locate the tab element fresh (avoid stale element reference)
                subsection = WebDriverWait(driver, 5).until(
                    EC.element_to_be_clickable((By.XPATH, f"{path_to_subsections}[normalize-space(text())='{section}']"))
                )
                subsection.click()

                # wait for the tab content to load (common wait for all subsections)
                WebDriverWait(driver, 5).until(
                    EC.presence_of_element_located(
                        (By.XPATH, "//table[contains(@class,'kv-grid-table')]/tbody/tr")
                    )
                )

                # call the right function for this subsection
                handler(driver, case_data)

            except Exception as e:
                print(f"⚠️ Skipped section {section}: {e}")

        data['cases'].append(case_data)
        driver.back()
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, "//table/tbody/tr"))
        )



In [37]:
driver = webdriver.Chrome()
driver.get("https://cases.shc.gov.pk/")
driver.implicitly_wait(10)


# Find all the district divs urls FIRST and store them
path = '//div[@class="row mt-4 mb-5"]/div[@class="col-md-2 mb-3"]/a'
items = driver.find_elements(By.XPATH, path)

print(f"Total items found: {len(items)}")
#items = items[:-2] #remove for now, causing errors,                            ------>>>>>>>>FIX LATER<<<<<<----------

# Store URLs and names FIRST before any navigation
district_data = []
for item in items:
    url = item.get_attribute('href')
    try:
        name = item.find_element(By.XPATH, './/div[@class="card-header"]').text
    except:
        continue
    district_data.append({'url': url, 'name': name})


print("District URLs found:")
for data in district_data:
    print(f"{data['name'].split(' ')[-1]}: {data['url']}")

# Now process each district
for district in district_data:
    district["name"]=district["name"].split(' ')[-1]
    filename = f'SindhCount_{district["name"]}.json'
    district_json = {
    "metadata":{
        "filename" :filename,
        "url" : district['url'],
    },
    "districts_list":{
        "sub_catagory":{
            "name":district['name'],
            "circuits":[]
        }
    }
}

    print(f"\n=== Processing {district['name']} ===")
    
    # Navigate to district page
    driver.get(district['url'])
    
    try:
        # Wait for the dropdown to load on the new page
        WebDriverWait(driver, 30).until(
            EC.presence_of_element_located((By.ID, "casessearch-circuitcode"))
        )
        
        # Find all options on THIS page
        options = driver.find_elements(By.XPATH, '//select[@id="casessearch-circuitcode"]/option')
        options = options[1:]  # Remove first placeholder option
        
        print(f"Number of court options found: {len(options)}")
        
        # Get values
        values = [option.get_attribute('value') for option in options]
        circuit_names = [option.text for option in options]
        print("Court values are:", values)
        
        # Process each sub court value
        for i, value in enumerate(values):
            try:
                print(f"Processing court value: {value}")
                circuit_name = circuit_names[i]
                print(f"Court name: {circuit_name}")
                
                # Set the value using JavaScript
                driver.execute_script(f"document.getElementById('casessearch-circuitcode').value = '{value}';")
                
                # Click search button
                search_button = WebDriverWait(driver, 30).until(
                    EC.element_to_be_clickable((By.ID, "submit_search"))
                )
                search_button.click()
                
                # Wait for results page
                WebDriverWait(driver, 30).until(
                    EC.presence_of_element_located((By.XPATH, "//table"))
                )
                print(f"Results page loaded for value {value}")
                
                circuit_data = {
                    "circuit_code": value,
                    "circuit_name": circuit_name,
                    "cases": []
                }
                
                # extract data from first page 
                try:
                    per_page_extraction(driver, circuit_data)
                    print("this is the circuit code", circuit_data)
                    district_json['districts_list']['sub_catagory']['circuits'].append(circuit_data)
                except Exception as e:
                    print(f"Error IN DATA EXTRACTION page for value {value}: {e}")
                    continue
                page_count=1

                # Check for pagination and navigate through pages
                while True:
                    try:       #                                                                      ------>>>>>>>>UNCOMMENT LATER<<<<<<----------
                        # try to find "next" button
                        next_buttons = driver.find_elements(By.CSS_SELECTOR, "li.next a")
                        page_count +=1
                        if not next_buttons or page_count>=2:                                      # STOP SCRAPING AFTER 30 PAGES FROM EACH SUBCOURT IN EACH MAIN COURT
                            print("✅ No next page button found. Stopping pagination.")
                            break

                        next_button = next_buttons[0]

                        # check if disabled
                        parent_li = next_button.find_element(By.XPATH, "./..")
                        if "disabled" in parent_li.get_attribute("class"):
                            print("✅ Reached the last page.")
                            break

                        # go to next page
                        next_button.click()
                        WebDriverWait(driver, 10).until(
                            EC.presence_of_element_located((By.XPATH, "//table/tbody/tr"))
                        )

                        per_page_extraction(driver, circuit_data)

                            
                    except Exception as e:
                        print(f"Error navigating pages: {e}")
                        break
                    

                # save data of each district
                with open(filename, 'w', encoding='utf-8') as f:
                    json.dump(district_json, f, indent=2)

                # Go back to the district search page
                driver.back()
                # Wait for the search page to reload
                WebDriverWait(driver, 30).until(
                    EC.presence_of_element_located((By.ID, "casessearch-circuitcode"))
                )
                print("Returned to district search page")
                
            except Exception as e:
                print(f"Error processing court value {value}: {e}")
                # Try to recover by going back to district page
                try:
                    driver.get(district['url'])
                    WebDriverWait(driver, 30).until(
                        EC.presence_of_element_located((By.ID, "casessearch-circuitcode"))
                    )
                except:
                    print("Could not recover, moving to next district")
                
            break  #                                                                      ------>>>>>>>>FIX LATER<<<<<<----------
    except Exception as e:
        print(f"Error loading district page {district['url']}: {e}")
        continue
    break #                                                                               ------>>>>>>>>FIX LATER<<<<<<----------
driver.quit()

Total items found: 6
District URLs found:
Karachi: https://cases.shc.gov.pk/khi
Sukkur: https://cases.shc.gov.pk/suk
Hyderabad: https://cases.shc.gov.pk/hyd
Larkana: https://cases.shc.gov.pk/lar
Mirpurkhas: https://cases.shc.gov.pk/mpkhas/web/index.php?r=cases/search-result&CasesSearch[CIRCUITCODE]=13
Sindh: https://cases.districtcourtssindh.gos.pk/

=== Processing Karachi ===
Number of court options found: 10
Court values are: ['1', '2', '3', '4', '5', '-1', '13', '14', '6', '7']
Processing court value: 1
Court name: Karachi
Results page loaded for value 1
herer1
herer2
herer3
herer1
herer2
herer3
herer1
herer2
herer3
herer1
herer2
herer3
herer1
herer2
herer3
herer1
herer2
herer3
herer1
herer2
herer3
herer1
herer2
herer3
herer1
herer2
herer3
herer1
herer2
herer3
herer1
herer2
herer3
herer1
herer2
herer3
herer1
herer2
herer3
herer1
herer2
herer3
herer1
herer2
herer3
herer1
herer2
herer3
herer1
herer2
herer3
herer1
herer2
herer3
herer1
herer2
herer3
herer1
herer2
herer3
this is the circ

In [17]:
data['name'].split(' ')[-1]

'Sindh'

In [20]:
district["name"].split(' ')[-1]

'Karachi'

In [ ]:
driver = webdriver.Chrome() # opens up the web page
driver.get("https://cases.shc.gov.pk/khi") # sindh high court website
driver.implicitly_wait(10) # waits for the web elements to load

# Direct XPath to find all option elements of the specific select
dropdown_path = '//select[@id="casessearch-circuitcode"]/option'
options = driver.find_elements(By.XPATH, dropdown_path)
options = options[1:]
print(f"Number of options found: {len(options)}")

# has the values to access each option
values = [option.get_attribute('value') for option in options]
# values = values[1:] # remove the first place holder value

print("Values are:", values)

for value in values:
    try:
        WebDriverWait(driver, 30).until(
            EC.presence_of_element_located((By.ID, "casessearch-circuitcode"))
        )
        driver.execute_script(f"document.getElementById('casessearch-circuitcode').value = '{value}';")
    
        # Click the search button
        search_button = WebDriverWait(driver, 30).until(
            EC.element_to_be_clickable((By.ID, "submit_search"))
        )
        search_button.click()
        
        print(f"Current URL: {driver.current_url}")
        WebDriverWait(driver, 30).until(
            EC.presence_of_element_located((By.XPATH, "//table"))
        )
        print(f"Results page loaded successfully for value {value}")


        # Go back to the search page
        driver.back()
        # Wait for search page to reload completely
        WebDriverWait(driver, 30).until(
            EC.presence_of_element_located((By.ID, "casessearch-circuitcode"))
        )
        print("Returned to search page")

    except Exception as e:
         print(f"Error setting value {value}: {e}")     






Number of options found: 10
Values are: ['1', '2', '3', '4', '5', '-1', '13', '14', '6', '7']
Current URL: https://cases.shc.gov.pk/khi/web/index.php?r=cases%2Fsearch
Results page loaded successfully
Returned to search page
Current URL: https://cases.shc.gov.pk/khi/web/index.php?r=cases%2Fsearch
Results page loaded successfully
Returned to search page
Current URL: https://cases.shc.gov.pk/khi/web/index.php?r=cases%2Fsearch
Results page loaded successfully
Returned to search page
Current URL: https://cases.shc.gov.pk/khi/web/index.php?r=cases%2Fsearch
Results page loaded successfully
Returned to search page
Current URL: https://cases.shc.gov.pk/khi/web/index.php?r=cases%2Fsearch
Results page loaded successfully
Returned to search page
Current URL: https://cases.shc.gov.pk/khi/web/index.php?r=cases%2Fsearch
Results page loaded successfully
Returned to search page
Current URL: https://cases.shc.gov.pk/khi/web/index.php?r=cases%2Fsearch
Results page loaded successfully
Returned to search p

'\nhttps://cases.shc.gov.pk/khi/web/index.php?r=cases%2Fsearch-result&CasesSearch%5BCASENO%5D=&CasesSearch%5BCASEYEAR%5D=&CasesSearch%5BCASENAMECODE%5D=&CasesSearch%5BBENCH%5D=&CasesSearch%5BCIRCUITCODE%5D=1&CasesSearch%5BMATTERCODE%5D=&CasesSearch%5BPARTY%5D=&CasesSearch%5BGOVT_AGENCY_CODE%5D=&CasesSearch%5BFIRNO%5D=&CasesSearch%5BFIRYEAR%5D=&CasesSearch%5BPOLICESTATIONCODE%5D=&CasesSearch%5BADVOCATECODE%5D=&CasesSearch%5BisPending%5D=&CasesSearch%5BisPending%5D=3 \nhttps://cases.shc.gov.pk/khi/web/index.php?r=cases%2Fsearch-result&CasesSearch%5BCASENO%5D=&CasesSearch%5BCASEYEAR%5D=&CasesSearch%5BCASENAMECODE%5D=&CasesSearch%5BBENCH%5D=&CasesSearch%5BCIRCUITCODE%5D=2&CasesSearch%5BMATTERCODE%5D=&CasesSearch%5BPARTY%5D=&CasesSearch%5BGOVT_AGENCY_CODE%5D=&CasesSearch%5BFIRNO%5D=&CasesSearch%5BFIRYEAR%5D=&CasesSearch%5BPOLICESTATIONCODE%5D=&CasesSearch%5BADVOCATECODE%5D=&CasesSearch%5BisPending%5D=&CasesSearch%5BisPending%5D=3      \nhttps://cases.shc.gov.pk/khi/web/index.php?r=cases%2Fs